# XAI Loan System - Interactive Demo

This notebook demonstrates the complete workflow of the XAI-driven Causal Counterfactual Loan System.

## Features Demonstrated:
1. **Loan Prediction**: Predict approval and interest rate
2. **SHAP Explanation**: Understand why the decision was made
3. **Counterfactual Generation**: Get actionable recommendations

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

sys.path.insert(0, '..')

from src.data import DataIngestionService, DataPreprocessor
from src.models import XGBoostLoanModel
from src.explainability import SHAPEngine, ExplanationReport
from src.counterfactual import CausalModel, CausalCFEngine
from src.ethics import ActionabilityScorer, FeasibilityGuard

print('System loaded successfully!')

## Setup: Load Data and Train Model

In [ ]:
# Load and preprocess data
ingestion = DataIngestionService()
data = ingestion.load_data(use_synthetic=True, sample_size=5000)

preprocessor = DataPreprocessor()
X, y = preprocessor.fit_transform(data)
X_train, X_val, X_test, y_train, y_val, y_test = preprocessor.split_data(X, y)

# Train model
model = XGBoostLoanModel(task='approval')
model.fit(X_train, y_train, X_val, y_val, verbose=False)

print(f'Model ROC-AUC: {model.evaluate(X_test, y_test)["roc_auc"]:.4f}')

In [ ]:
# Initialize explanation and counterfactual engines
shap_engine = SHAPEngine(model, X_train)
shap_engine.initialize(X_train)

causal_model = CausalModel()
X_with_target = X.copy()
X_with_target['loan_status'] = y.values
causal_model.estimate_from_data(X_with_target)

scorer = ActionabilityScorer()
guard = FeasibilityGuard()

cf_engine = CausalCFEngine(
    model=model,
    causal_model=causal_model,
    feature_names=list(X_train.columns),
    actionability_scorer=scorer,
)

print('All engines initialized!')

## Demo: Complete Workflow

Simulate a loan applicant and go through the full explanation pipeline.

In [ ]:
# Select a test applicant who was rejected
predictions = model.predict(X_test)
rejected_idx = X_test[predictions == 0].index[0]
applicant = X_test.loc[[rejected_idx]]

print('=== APPLICANT PROFILE ===')
key_features = ['annual_inc', 'dti', 'fico_score', 'revol_util', 'emp_length', 'loan_amnt']
for f in key_features:
    if f in applicant.columns:
        print(f'{f:20s}: {applicant[f].values[0]:>10.2f}')

In [ ]:
# Step 1: Get prediction
approval_proba = model.predict_proba(applicant)[0]
approved = model.predict(applicant)[0] == 1

print('=== LOAN DECISION ===')
print(f'Approval Probability: {approval_proba[1]:.1%}')
print(f'Decision: {"APPROVED" if approved else "NEEDS IMPROVEMENT"}')

In [ ]:
# Step 2: SHAP Explanation (Diagnostic - Why?)
explanation = shap_engine.explain_instance(applicant, top_k=5)

print('\n=== DIAGNOSTIC EXPLANATION (Why this decision?) ===')
print('Top factors influencing the decision:')

for feat in explanation['top_features'][:5]:
    direction = 'HELPS' if feat['shap_value'] > 0 else 'HURTS'
    print(f"  {feat['feature']:20s}: {direction} (impact: {abs(feat['shap_value']):.3f})")

In [ ]:
# Step 3: Counterfactual Generation (Prescriptive - What to do?)
cf_results = cf_engine.generate_counterfactuals(applicant, num_cfs=3)

print('\n=== ACTIONABLE RECOMMENDATIONS (What to change?) ===')

if cf_results['success']:
    print(f'Generated {len(cf_results["counterfactuals"])} options:')
    
    original = applicant.iloc[0].to_dict()
    
    for i, cf_data in enumerate(cf_results['counterfactuals']):
        cf = cf_data['cf_features']
        
        print(f'\n--- Option {i+1} (Actionability: {cf_data["actionability_score"]:.2f}) ---')
        
        for f in key_features:
            if f in cf and f in original:
                if abs(cf[f] - original[f]) > 0.01:
                    direction = 'Increase' if cf[f] > original[f] else 'Decrease'
                    print(f"  {direction} {f}: {original[f]:.2f} -> {cf[f]:.2f}")
else:
    print('Could not generate recommendations.')

In [ ]:
# Step 4: Detailed Improvement Suggestions
if cf_results['success'] and cf_results['counterfactuals']:
    best_cf = cf_results['counterfactuals'][0]
    suggestions = scorer.get_improvement_suggestions(original, best_cf['cf_features'])
    
    print('\n=== DETAILED IMPROVEMENT PLAN ===')
    for s in suggestions[:5]:
        print(f"\n{s['action']}")
        print(f"  Current: {s['from_value']:.2f}")
        print(f"  Target:  {s['to_value']:.2f}")
        print(f"  Difficulty: {s['difficulty']}")
        print(f"  Estimated time: {s['estimated_time']}")

## Summary

The XAI Loan System provides:

1. **Prediction**: Accurate loan approval predictions
2. **Diagnosis (SHAP)**: Understanding why decisions are made
3. **Prescription (Causal CF)**: Actionable recommendations that respect causal constraints

Key advantages of our causal counterfactual approach:
- Recommendations respect real-world causal dependencies
- Actionability scores quantify feasibility
- Ethics layer ensures fairness and constraint compliance